# acspype - Ship Flowthrough Example
This notebook demonstrates how to process ACS data using the `acspype` package with the assistance of co-located ancillary data products.

The dataset used in this example is from a cruise aboard the NOAA Ship Shimada that occurred in May 2024. The example data has been roughly merged on common timestamps using nearest neighbors to demonstrate the functionality of `acpype`. Time lag correction for flowthrough data has not been performed, and is a practice that is recommended when reviewing data at finer resolutions.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
import xarray as xr


from acspype import ACSTSCor
import acspype.processing as acsproc
import acspype.qaqc as acsqaqc

from acspype.tutorial import load_shimada_tutorial_data, load_shimada_tutorial_device_file

## Load Tutorial Data and Device File

In [2]:
ds = load_shimada_tutorial_data()
dev = load_shimada_tutorial_device_file()  # Normally you would use ACSDev(filepath)

In [4]:
ds

<xarray.Dataset> Size: 84MB
Dimensions:                            (time: 28547, a_wavelength: 89,
                                        c_wavelength: 89)
Coordinates:
  * time                               (time) datetime64[ns] 228kB 2024-06-03...
  * a_wavelength                       (a_wavelength) float64 712B 401.9 ... ...
  * c_wavelength                       (c_wavelength) float64 712B 401.4 ... ...
Data variables: (12/13)
    latitude                           (time) float64 228kB ...
    longitude                          (time) float64 228kB ...
    raw_internal_temperature           (time) int64 228kB ...
    raw_external_temperature           (time) int64 228kB ...
    a_signal                           (time, a_wavelength) int64 20MB ...
    a_reference                        (time, a_wavelength) int64 20MB ...
    ...                                 ...
    c_reference                        (time, c_wavelength) int64 20MB ...
    elapsed_time                       (time) int64 228kB ...
    sea_water_temperature              (time) float64 228kB ...
    sea_water_electrical_conductivity  (time) float64 228kB ...
    sea_water_practical_salinity       (time) float64 228kB ...
    seawater_state                     (time) float64 228kB ...
Attributes: (12/13)
    acs_serial_number:           ACS412
    acs_device_file:             ACS0412_2023-05-10.DEV
    acs_num_output_wavelengths:  89
    flow_meter_serial_number:    11222239
    flowmeter_kfactor:           617.2
    flowmeter_manufacturer:      Omega
    ...                          ...
    tsg_serial_number:           tsg00001
    tsg_manufacturer:            Atlas Scientific
    tsg_model:                   Industrial Conductivity Probe K1.0
    valve_serial_number:         valve00001
    dataset_description:         This is a curated dataset of ACS and ancilla...
    dataset_owner:               Maria T. Kavanaugh, Oregon State University

In [5]:
dev.to_xarray()

<xarray.Dataset> Size: 53kB
Dimensions:          (a_wavelength: 89, c_wavelength: 89, temperature_bin: 35)
Coordinates:
  * a_wavelength     (a_wavelength) float64 712B 401.9 403.9 ... 738.5 742.3
  * c_wavelength     (c_wavelength) float64 712B 401.4 403.9 ... 738.1 741.8
  * temperature_bin  (temperature_bin) float64 280B 0.8352 1.201 ... 33.5 34.52
Data variables:
    a_offset         (a_wavelength) float64 712B -0.04384 0.01841 ... -0.2669
    a_delta_t        (a_wavelength, temperature_bin) float64 25kB -0.004055 ....
    c_offset         (c_wavelength) float64 712B 0.9704 1.075 ... -0.7509
    c_delta_t        (c_wavelength, temperature_bin) float64 25kB 0.01514 ......
Attributes: (12/25)
    device_filepath:                C:\Users\Ian\phd\projects\acspype\acspype...
    sensor_type:                    ACS Meter
    serial_number_hexdec:           5300019C
    serial_number:                  ACS-00412
    device_file_structure_version:  3
    tcal:                           22.5
    ...                             ...
    max_c_difference:               0.0
    min_a_counts:                   0.0
    min_c_counts:                   0.0
    min_r_counts:                   0.0
    max_temp_sd:                    0.0
    max_depth_sd:                   0.0

In [8]:
np.all(dev.a_wavelength) == np.all(ds.a_wavelength)

<xarray.DataArray 'a_wavelength' ()> Size: 1B
array(True)

## Load TS Correction Coefficients

In [3]:
tscor = ACSTSCor()

## Reprocess the Data

### Compute Internal and External Temperature

In [4]:
ds['internal_temperature'] = acsproc.compute_internal_temperature(ds['raw_internal_temperature'])
ds['external_temperature'] = acsproc.compute_external_temperature(ds['raw_external_temperature'])

## Compute Uncorrected Absorption and Attenuation

In [5]:
ds['a_uncorrected'] = acsproc.compute_uncorrected(ds['a_signal'], ds['a_reference'], dev)
ds['c_uncorrected'] = acsproc.compute_uncorrected(ds['c_signal'], ds['c_reference'], dev)

## Compute Measured Absorption and Attenuation
Note that these values are called a_m_discontinuity and c_m_discontinuity in the code below. This is because these spectra contain an observed discontinuity somewhere between 530 and 600 nm. The discontinuity is corrected in the next cell.

In [6]:
ds['a_m_discontinuity'] = acsproc.compute_measured(ds['a_uncorrected'], 'a', ds['internal_temperature'], dev)
ds['c_m_discontinuity'] = acsproc.compute_measured(ds['c_uncorrected'], 'c', ds['internal_temperature'], dev)

In [7]:
discontinuity_index = acsproc.find_discontinuity_index(ds['a_wavelength'], ds['c_wavelength'])
ds['a_m'],ds['a_discontinuity_offset'] = acsproc.discontinuity_correction(ds.a_m_discontinuity, 'a_wavelength', discontinuity_index)
ds['c_m'],ds['c_discontinuity_offset'] = acsproc.discontinuity_correction(ds.c_m_discontinuity, 'c_wavelength', discontinuity_index)

## Correct Measured Values for the Effects of Temperature and Salinity

In [8]:
ds['a_mts'] = acsproc.ts_correction(ds.a_m, 'a', ds.sea_water_temperature, ds.sea_water_practical_salinity, dev, tscor)
ds['c_mts'] = acsproc.ts_correction(ds.c_m, 'c', ds.sea_water_temperature, ds.sea_water_practical_salinity, dev, tscor)

## Flag Data

In [10]:
ds['flag_elapsed_time'] = acsqaqc.elapsed_time_test(ds['elapsed_time'], fail_threshold=60*1000, suspect_threshold=3*60*1000)
ds['flag_internal_temperature'] = acsqaqc.internal_temperature_test(ds['internal_temperature'], dev)

ds['flag_inf_nan_a_uncorrected'] = acsqaqc.inf_nan_test(ds['a_uncorrected'])
ds['flag_inf_nan_c_uncorrected'] = acsqaqc.inf_nan_test(ds['c_uncorrected'])

## Remove Poor Data

In [11]:
ds = ds.where(ds.flag_elapsed_time != 4, drop = True) # Drop any and all data where the elapsed time does not pass the elapsed time test.
ds = ds.where(ds.flag_internal_temperature != 4, drop = True)  # Drop any and all data where the internal temperature of the sensor exceeded the calibration range defined in the device file.
ds = ds.where(ds.flag_inf_nan_a_uncorrected != 4, drop = True)  # Drop any and all data where the a_uncorrected signal is NaN or Inf.
ds = ds.where(ds.flag_inf_nan_c_uncorrected != 4, drop = True)  # Drop any and all data where the c_uncorrected signal is NaN or Inf.

For the Shimada 202405 tutorial data in acpype, a subset of the data was chosen in the middle of the cruise. The ACS deployed was also a newer ACS with the new LED light sources. This is probably why no data are flagged as poor and removed

## Interpolate to Common Wavelengths

In [ ]:
ds = acsproc.interpolate_common_wavelengths(ds,step = 1, wavelength_range='infer')

## Apply Proportional Scattering Correction

In [ ]:
ds['a_mts_proportional]'] = acsproc.scattering_correction_proportional(ds.a_mts,ds.c_mts,reference_wavelength = 715) # Method 3, requires common wavelength bins. 

## Split Data

In [ ]:
dissolved = ds.where(ds.seawater_state == 1, drop = True)
total = ds.where(ds.seawater_state == 0, drop = True)

### Calculating Particulate Fraction

In [ ]:
dissolved_r = dissolved.rolling({'time': 4 *3 + 1}, center=True, min_periods = 1).median(skipna = True)  # Apply a boxcar filter
dissolved_r = dissolved_r.resample({'time': '1min'}).mean(skipna = True)  # Resample to 1 minute average bins.
dissolved_ri = dissolved_r.interp(time = total.time, method = 'nearest')  # Interpolate to total time bins.
dissolved_ri = dissolved_ri.interpolate_na(dim = 'time', method = 'linear')  # Interpolate any NaN values in the time dimension.
total_r = total.rolling({'time': 4 *3 + 1}, center=True).median(skipna = True)  # Apply a boxcar filter
particulate = total_r - dissolved_ri
particulate = particulate.dropna(dim='time', how='all')  # Drop any time bins where all variables are NaN.

### Single Spectra Plots

In [ ]:
dis_spec = dissolved.sel(time = dissolved.time.values[-1000])
spec_time = dis_spec.time.values - np.timedelta64(10*60, 's')
tot_spec = total.sel(time = spec_time, method = 'nearest')
par_spec = particulate.sel(time = tot_spec.time, method = 'nearest')

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (12,10), constrained_layout=True, sharex = True, sharey = True)

ax[0].plot(tot_spec.wavelength, tot_spec.a_mts, label = 'no scattering correction',linewidth = 2)
ax[0].plot(tot_spec.wavelength, tot_spec.a_mts_baseline, label = 'baseline scattering correction',linewidth = 2)
ax[0].plot(tot_spec.wavelength, tot_spec.a_mts_fixed, label = 'fixed scattering correction',linewidth = 2)
ax[0].plot(tot_spec.wavelength, tot_spec.a_mts_proportional, label = 'proportional scattering correction',linewidth = 2)
ax[0].set_title(f'Total Spectra\n{tot_spec.time.values.astype(str)[:-3]}')



ax[1].plot(dis_spec.wavelength, dis_spec.a_mts, label = 'no scattering correction',linewidth = 2)
ax[1].plot(dis_spec.wavelength, dis_spec.a_mts_baseline, label = 'baseline scattering correction',linewidth = 2)
ax[1].plot(dis_spec.wavelength, dis_spec.a_mts_fixed, label = 'fixed scattering correction',linewidth = 2)
ax[1].plot(dis_spec.wavelength, dis_spec.a_mts_proportional, label = 'proportional scattering correction',linewidth = 2)
ax[1].set_title(f'Dissolved Spectra\n{dis_spec.time.values.astype(str)[:-3]}')


ax[2].plot(par_spec.wavelength, par_spec.a_mts, label = 'no scattering correction',linewidth = 2)
ax[2].plot(par_spec.wavelength, par_spec.a_mts_baseline, label = 'baseline scattering correction',linewidth = 2)
ax[2].plot(par_spec.wavelength, par_spec.a_mts_fixed, label = 'fixed scattering correction',linewidth = 2)
ax[2].plot(par_spec.wavelength, par_spec.a_mts_proportional, label = 'proportional scattering correction',linewidth = 2)
ax[2].set_title(f'Particulate Spectra\n{par_spec.time.values.astype(str)[:-3]}')
ax[2].legend(loc = 'upper right')


ax[-1].xaxis.set_major_locator(MultipleLocator(50))
ax[-1].xaxis.set_minor_locator(MultipleLocator(5))
ax[-1].set_ylim(-0.05, 0.24)
ax[-1].yaxis.set_major_locator(MultipleLocator(0.02))
ax[-1].yaxis.set_minor_locator(MultipleLocator(0.01))


ax[0].set_ylabel(r'Absorption ($\frac{1}{m}$)')
ax[1].set_xlabel('Wavelength (nm)')

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (12,10), constrained_layout=True, sharex = True, sharey = True)

x, y = np.meshgrid(total.time, total.wavelength)
z = total.a_mts_proportional
ax[0].pcolormesh(x,y, z.T, cmap='viridis', vmin = 0, vmax = 0.19)

x, y = np.meshgrid(dissolved.time, dissolved.wavelength)
z = dissolved.a_mts_proportional
ax[1].pcolormesh(x,y, z.T, cmap='viridis', vmin =0, vmax = 0.19)

x, y = np.meshgrid(particulate.time, particulate.wavelength)
z = particulate.a_mts_proportional
ax[2].pcolormesh(x,y, z.T, cmap='viridis', vmin =0, vmax = 0.19)
